# Bronze Layer
In this notebook, we will create the Bronze Layer of our Data Pipeline. We already have fetched the data from the APIs, so now we will perform the initial cleanups.
* Insert the *ticker* column for each of the datasets
* Transform the date column from string to datetime
* Drop any data before 2019-01-01, as we want to focus on the last 5 years of data.
* Union all the data together in a single table


In [0]:
# Importing needed Pyspark functions
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, sum, when
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, DateType

In [0]:
def transform_data(ticker):
    '''
    Function that (1) loads a dataset; 
    (2) adds a ticker column for the stock dataset; 
    (3) Transform the column to datetime; 
    (4) Drop data before 2019-01-01
    Inputs:
    * ticker: str = code to be added to all obervations
    '''

    # File path to be loaded
    path = f'/FileStore/tables/raw/{ticker}'

    # As the dataset for the ETF "DJUSTL" was pulled from a different API, it has the column date named as datetime. It needs to be corrected
    if ticker == "DJUSTL":
        df = (
            spark.read.parquet(path)
            .withColumn('ticker', F.lit(ticker))
            .withColumnRenamed('datetime', 'date')
        )
    else:    
        # (2) Add ticker column
        df = (
            spark.read.parquet(path)
            .withColumn('ticker', F.lit(ticker))
        )

    # Steps (3) and (4)
    # If working with economic indices, adapt for less columns
    if ticker in ['INFLATION', 'REAL_GDP_PER_CAPITA', 'CPI']:
        df = (df
            .select( 'ticker', 'value',
                      F.to_date('date').alias('date')  )
            .filter( col('date') >= '2019-01-01' ) #data cleanup, drop data before 2019
          )
    else:
        df = (df
            .select( 'ticker', 'open', 'high', 'low', 'close', 'volume',
                      F.to_date('date').alias('date')  )
            .filter( col('date') >= '2019-01-01' ) #data cleanup, drop data before 2019
          )

    # Return transformed data    
    return df

In [0]:
# Function to add RSI to the stock dataset
def add_RSI(df, ticker):
    '''Function to add RSI column to the stocks
    inputs:
    ticker: str = stock code'''

    # File path to be loaded
    path = f'/FileStore/tables/raw/RSI/{ticker}'
    rsi = spark.read.parquet(path)

    # Transform date to datetime format
    rsi = (rsi
            .select( F.to_date('date').alias('date'),
                    col('value').alias('RSI')  )
            .filter( col('date') >= '2019-01-01' ) #data cleanup, drop data before 2019
    )

    # Add RSI to the dataset
    df = (df
          .join(rsi, on='date', how= 'inner')
          )

    # Return transformed data
    return df


In [0]:
names = ['CHTR' , 'CMCSA', 'T', 'TMUS', 'VZ']

# Save transformed datasets
for folder in names:
    stock = transform_data(ticker = folder)
    stock_bronze = add_RSI(stock, ticker=folder)
    path = f'/FileStore/tables/bronze/{folder}' 
    stock_bronze.coalesce(1).write.format('delta').mode('overwrite').save(path)
    print(f'{folder} transformed')


indicators = ['INFLATION', 'REAL_GDP_PER_CAPITA', 'CPI', 'DJUSTL']
# Save transformed datasets
for folder in indicators:
    df_ind = transform_data(ticker = folder)
    path = f'/FileStore/tables/bronze/{folder}' 
    df_ind.coalesce(1).write.format('parquet').mode('overwrite').save(path)
    print(f'{folder} transformed')

CHTR transformed
CMCSA transformed
T transformed
TMUS transformed
VZ transformed
INFLATION transformed
REAL_GDP_PER_CAPITA transformed
CPI transformed
DJUSTL transformed


In [0]:
t = spark.read.parquet('/FileStore/tables/bronze/TMUS')
t.display()

date,ticker,open,high,low,close,volume,RSI
2024-01-05,TMUS,162.25,163.445,161.74,163.37,4544731,78.3763
2024-01-04,TMUS,162.85,163.57,162.1801,162.3,4068757,76.4368
2024-01-03,TMUS,163.0,164.499,161.87,162.08,4608645,76.0263
2024-01-02,TMUS,160.33,163.0,160.33,161.91,5049655,75.7228
2023-12-29,TMUS,158.85,160.882,158.37,160.33,3868051,72.7452
2023-12-28,TMUS,158.17,159.3098,157.745,158.95,3013150,69.7346
2023-12-27,TMUS,156.02,158.52,155.81,157.94,3562928,67.2781
2023-12-26,TMUS,155.11,157.28,153.84,156.83,2991219,64.3231
2023-12-22,TMUS,155.16,156.18,154.32,155.71,2783959,61.0253
2023-12-21,TMUS,155.25,155.39,154.27,154.93,3213592,58.5474
